In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/stats207

/content/drive/My Drive/Colab Notebooks/stats207


In [ ]:
import numpy as np 
import pandas as pd
import torch
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader 
import sklearn.metrics as metrics
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler,  FunctionTransformer

cites:


# Data Pre-Processing

In [ ]:
aqi_daily = pd.read_csv('aqi_daily_cleaned.csv')
aqi_daily.head()

,Date,Overall.AQI.Value,Main.Pollutant,Site.Name..of.Overall.AQI.,CO,Ozone,PM25,NO2
0,2012-01-01,83,PM2.5,San Jose - Jackson,14,22,83,31
1,2012-01-02,75,PM2.5,San Jose - Jackson,16,19,75,35
2,2012-01-03,93,PM2.5,San Jose - Jackson,17,8,93,33
3,2012-01-04,97,PM2.5,San Jose - Jackson,20,19,97,42
4,2012-01-05,65,PM2.5,San Jose - Jackson,20,18,65,48


In [ ]:
aqi_daily.tail()

,Date,Overall.AQI.Value,Main.Pollutant,Site.Name..of.Overall.AQI.,CO,Ozone,PM25,NO2
3648,2021-12-27,33,Ozone,San Martin,6,33,17,23
3649,2021-12-28,32,Ozone,San Martin,6,32,20,24
3650,2021-12-29,27,Ozone,San Martin,7,27,17,25
3651,2021-12-30,24,NO2,San Jose - Knox Avenue,6,22,23,24
3652,2021-12-31,50,PM2.5,San Jose - Knox Avenue,9,28,50,24


### Time Feature Processing

In [ ]:
aqi_daily['Date'] = pd.to_datetime(aqi_daily['Date'])

In [ ]:
aqi_daily['day_of_month'] = aqi_daily.Date.dt.day - 1 # ensure indexing starts from zero 
aqi_daily['month'] = aqi_daily.Date.dt.month - 1
aqi_daily['day_of_week'] = aqi_daily.Date.dt.weekday

In [ ]:
# Sine/cosine transformation 
def sin_transformer(period):
	return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))

def cos_transformer(period):
	return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

In [ ]:
aqi_daily["day_of_month_sin"] = sin_transformer(12).fit_transform(aqi_daily['day_of_month'])
aqi_daily["day_of_month_cos"] = cos_transformer(12).fit_transform(aqi_daily['day_of_month'])

aqi_daily["month_sin"] = sin_transformer(12).fit_transform(aqi_daily['month'])
aqi_daily["month_cos"] = cos_transformer(12).fit_transform(aqi_daily['month'])

aqi_daily["day_of_week_sin"] = sin_transformer(12).fit_transform(aqi_daily['day_of_week'])
aqi_daily["day_of_week_cos"] = cos_transformer(12).fit_transform(aqi_daily['day_of_week'])

In [ ]:
aqi_daily_subset = aqi_daily.drop(['Date',
 'Main.Pollutant',
 'Site.Name..of.Overall.AQI.',
 'day_of_month',
 'month',
 'day_of_week'], axis=1)

In [ ]:
# Split Train and Test 
train = aqi_daily_subset.iloc[0:2922, :].values
test = aqi_daily_subset.iloc[2922:, :].values

print(train.shape)
print(test.shape)

(2922, 11)
(731, 11)


In [ ]:
train[0]

array([ 8.3000000e+01,  1.4000000e+01,  2.2000000e+01,  8.3000000e+01,
        3.1000000e+01,  0.0000000e+00,  1.0000000e+00,  0.0000000e+00,
        1.0000000e+00,  1.2246468e-16, -1.0000000e+00])

In [ ]:
# Normalize Input 
scaler = MinMaxScaler()
# scaler = StandardScaler() 

train[:, :5] = scaler.fit_transform(train[:, :5])
test[:, :5] = scaler.fit_transform(test[:, :5] )

In [ ]:
train[0]

array([ 3.90109890e-01,  4.61538462e-01,  1.20879121e-01,  4.18848168e-01,
        3.25301205e-01,  0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
        1.00000000e+00,  1.22464680e-16, -1.00000000e+00])

# Dataset

In [ ]:
timesteps = 3
n_cols = 11

In [ ]:
def create_sequences(data, timesteps, n_cols):
  data_timesteps = np.array([[j for j in data[i:i+timesteps+1]] for i in range(0,len(data)-timesteps)])[:,:,]
  x, y = data_timesteps[:,:-1, :], data_timesteps[:,-1:, 0]

  return x, y 

In [ ]:
x_train, y_train = create_sequences(train, 3, 11)

In [ ]:
x_train[:4, ]

array([[[ 3.90109890e-01,  4.61538462e-01,  1.20879121e-01,
          4.18848168e-01,  3.25301205e-01,  0.00000000e+00,
          1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
          1.22464680e-16, -1.00000000e+00],
        [ 3.46153846e-01,  5.38461538e-01,  1.04395604e-01,
          3.76963351e-01,  3.73493976e-01,  5.00000000e-01,
          8.66025404e-01,  0.00000000e+00,  1.00000000e+00,
          0.00000000e+00,  1.00000000e+00],
        [ 4.45054945e-01,  5.76923077e-01,  4.39560440e-02,
          4.71204188e-01,  3.49397590e-01,  8.66025404e-01,
          5.00000000e-01,  0.00000000e+00,  1.00000000e+00,
          5.00000000e-01,  8.66025404e-01]],

       [[ 3.46153846e-01,  5.38461538e-01,  1.04395604e-01,
          3.76963351e-01,  3.73493976e-01,  5.00000000e-01,
          8.66025404e-01,  0.00000000e+00,  1.00000000e+00,
          0.00000000e+00,  1.00000000e+00],
        [ 4.45054945e-01,  5.76923077e-01,  4.39560440e-02,
          4.71204188e-01,  3.49397590e-01,

In [ ]:
y_train[0]

array([0.46703297])

In [ ]:
x_train[0]

array([[ 3.90109890e-01,  4.61538462e-01,  1.20879121e-01,
         4.18848168e-01,  3.25301205e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         1.22464680e-16, -1.00000000e+00],
       [ 3.46153846e-01,  5.38461538e-01,  1.04395604e-01,
         3.76963351e-01,  3.73493976e-01,  5.00000000e-01,
         8.66025404e-01,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00],
       [ 4.45054945e-01,  5.76923077e-01,  4.39560440e-02,
         4.71204188e-01,  3.49397590e-01,  8.66025404e-01,
         5.00000000e-01,  0.00000000e+00,  1.00000000e+00,
         5.00000000e-01,  8.66025404e-01]])

In [ ]:
print(x_train[0].shape, y_train[0].shape)

(3, 11) (1,)


In [ ]:
class TimeSeries(Dataset):
    def __init__(self,data, timesteps, n_features):
        # create sequences of length timesteps, including n_features for each item in the sequence 
        data_timesteps = np.array([[j for j in data[i:i+timesteps+1]] for i in range(0,len(data)-timesteps)])[:,:,]
        x, y = data_timesteps[:,:-1, :], data_timesteps[:,-1:, 0]

        self.x = torch.tensor(x,dtype=torch.float32)
        self.y = torch.tensor(y,dtype=torch.float32)
        self.len = len(x)

    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]
  
    def __len__(self):
        return self.len

# GRU 

In [ ]:
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(GRU, self).__init__()

        self.layer_dim = layer_dim
        self.hidden_dim = hidden_dim

        self.gru = nn.GRU(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.gru(x)
        out = out[:, -1]
        out = self.fc(out)
        return out

# CNN-LSTM

In [ ]:
x_train[0].T

array([[ 3.90109890e-01,  3.46153846e-01,  4.45054945e-01],
       [ 4.61538462e-01,  5.38461538e-01,  5.76923077e-01],
       [ 1.20879121e-01,  1.04395604e-01,  4.39560440e-02],
       [ 4.18848168e-01,  3.76963351e-01,  4.71204188e-01],
       [ 3.25301205e-01,  3.73493976e-01,  3.49397590e-01],
       [ 0.00000000e+00,  5.00000000e-01,  8.66025404e-01],
       [ 1.00000000e+00,  8.66025404e-01,  5.00000000e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 1.22464680e-16,  0.00000000e+00,  5.00000000e-01],
       [-1.00000000e+00,  1.00000000e+00,  8.66025404e-01]])

In [ ]:
(torch.transpose(torch.tensor(x_train), dim0=1,dim1=2))[0]

tensor([[ 3.9011e-01,  3.4615e-01,  4.4505e-01],
        [ 4.6154e-01,  5.3846e-01,  5.7692e-01],
        [ 1.2088e-01,  1.0440e-01,  4.3956e-02],
        [ 4.1885e-01,  3.7696e-01,  4.7120e-01],
        [ 3.2530e-01,  3.7349e-01,  3.4940e-01],
        [ 0.0000e+00,  5.0000e-01,  8.6603e-01],
        [ 1.0000e+00,  8.6603e-01,  5.0000e-01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 1.2246e-16,  0.0000e+00,  5.0000e-01],
        [-1.0000e+00,  1.0000e+00,  8.6603e-01]], dtype=torch.float64)

In [ ]:
c = nn.Conv1d(in_channels=11, out_channels=11, kernel_size=2, stride=1)
c = c.double()
input = torch.transpose(torch.tensor(x_train), dim0=1,dim1=2) 
input = input.double() 
output = c(input)
output

tensor([[[-0.4128, -0.0472],
         [ 0.0374, -0.1242],
         [ 0.0945, -0.1356],
         ...,
         [ 0.3332,  0.1691],
         [-0.0808, -0.2248],
         [ 0.5766,  0.4601]],

        [[-0.0472,  0.1541],
         [-0.1242,  0.0049],
         [-0.1356, -0.2768],
         ...,
         [ 0.1691,  0.2953],
         [-0.2248, -0.1074],
         [ 0.4601,  0.2472]],

        [[ 0.1541,  0.2681],
         [ 0.0049,  0.1774],
         [-0.2768, -0.3899],
         ...,
         [ 0.2953,  0.3627],
         [-0.1074,  0.0348],
         [ 0.2472, -0.0650]],

        ...,

        [[-0.0755, -0.0840],
         [ 0.0447,  0.2479],
         [-0.2889, -0.4055],
         ...,
         [ 0.1212,  0.2284],
         [-0.1297,  0.2378],
         [ 0.3006,  0.1268]],

        [[-0.0840, -0.1955],
         [ 0.2479,  0.4106],
         [-0.4055, -0.3959],
         ...,
         [ 0.2284,  0.3408],
         [ 0.2378,  0.5532],
         [ 0.1268, -0.0686]],

        [[-0.1955, -0.3832],
       

In [ ]:
class CNNLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob, window, n_filters):
        super(CNNLSTM, self).__init__()

        self.layer_dim = layer_dim
        self.hidden_dim = hidden_dim

        self.conv1d = nn.Conv1d(in_channels=input_dim, out_channels=n_filters, kernel_size = window, stride = 1) # TODO: TUNE THIS 

        self.lstm = nn.LSTM(
            n_filters, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.transpose(x, dim0=1, dim1=2)
        out = self.conv1d(x)
        out = torch.transpose(out, dim0=1, dim1=2)
        out, _ = self.lstm(out)
        out = out[:, -1]
        out = self.fc(out)
        return out

# Train 

In [ ]:
# TODO: use GPU 

In [ ]:
N_FEATURES = train.shape[1]
INPUT_DIM = N_FEATURES
OUTPUT_DIM = 1

In [ ]:
# GRU Hyperparameters
TIMESTEPS = 10
LEARNING_RATE = 0.001
N_EPOCHS = 100
BATCH_SIZE = 32
DROPOUT = 0 # TODO: CHANGE LATER
WEIGHT_DECAY = 0 # TODO: CHANGE LATER
HIDDEN_DIM = 128
LAYER_DIM = 3

In [ ]:
# Additional CNN-LSTM Hyperparameters 
WINDOW = 3
N_FILTERS = N_FEATURES 

In [ ]:
train_ts = TimeSeries(train, TIMESTEPS, N_FEATURES)
test_ts = TimeSeries(test, TIMESTEPS, N_FEATURES)

In [ ]:
train_loader = DataLoader(train_ts, shuffle=True, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_ts, shuffle=True, batch_size=BATCH_SIZE)

In [ ]:
 model = CNNLSTM(INPUT_DIM, HIDDEN_DIM, LAYER_DIM, OUTPUT_DIM, DROPOUT, WINDOW, N_FILTERS)

In [ ]:
# model = GRU(INPUT_DIM, HIDDEN_DIM, LAYER_DIM, OUTPUT_DIM, DROPOUT) 

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=LEARNING_RATE)

In [ ]:
def train_model(model, train_loader, test_loader, criterion, optimizer, 
          num_epochs=N_EPOCHS, learning_rate=LEARNING_RATE, verbose=True):

    for epoch in range(1, num_epochs + 1):
        batch_num = 1
        for inputs, targets in train_loader:  

            # If using GPU 
            if torch.cuda.is_available():
                model = model.cuda()
                inputs = inputs.cuda()
                targets = targets.cuda()

            optimizer.zero_grad() 
            output = model(inputs)
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step() 

            if verbose and batch_num % 10 == 0:  # Print every 5 batches                              
                print(f'Epoch [{epoch}/{num_epochs}], Step [{batch_num}/{len(train_loader)}], '
                      f'Loss: {loss.item():.4f}')
            
            batch_num += 1

In [ ]:
train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=N_EPOCHS, learning_rate=LEARNING_RATE)

Epoch [1/100], Step [10/91], Loss: 0.0058
Epoch [1/100], Step [20/91], Loss: 0.0104
Epoch [1/100], Step [30/91], Loss: 0.0097
Epoch [1/100], Step [40/91], Loss: 0.0126
Epoch [1/100], Step [50/91], Loss: 0.0049
Epoch [1/100], Step [60/91], Loss: 0.0256
Epoch [1/100], Step [70/91], Loss: 0.0111
Epoch [1/100], Step [80/91], Loss: 0.0248
Epoch [1/100], Step [90/91], Loss: 0.0201
Epoch [2/100], Step [10/91], Loss: 0.0049
Epoch [2/100], Step [20/91], Loss: 0.0079
Epoch [2/100], Step [30/91], Loss: 0.0045
Epoch [2/100], Step [40/91], Loss: 0.0150
Epoch [2/100], Step [50/91], Loss: 0.0030
Epoch [2/100], Step [60/91], Loss: 0.0082
Epoch [2/100], Step [70/91], Loss: 0.0051
Epoch [2/100], Step [80/91], Loss: 0.0045
Epoch [2/100], Step [90/91], Loss: 0.0055
Epoch [3/100], Step [10/91], Loss: 0.0053
Epoch [3/100], Step [20/91], Loss: 0.0082
Epoch [3/100], Step [30/91], Loss: 0.0077
Epoch [3/100], Step [40/91], Loss: 0.0083
Epoch [3/100], Step [50/91], Loss: 0.0064
Epoch [3/100], Step [60/91], Loss:

# Evaluation

In [ ]:
def evaluate(test_loader,batch_size, n_features):
  with torch.no_grad():
      predictions = []
      values = []

      for x_test, y_test in test_loader:
        batch_size = x_test.shape[0]
        x_test = x_test.view([batch_size, -1, n_features])
        y_preds = model(x_test)
        y_preds = [ j for i in y_preds.tolist() for j in i  ]
        y_test = [ j for i in y_test.tolist() for j in i  ]
        predictions.extend(y_preds)
        values.extend(y_test)

  return predictions, values

In [ ]:
y_test_preds, y_test = evaluate(test_loader, BATCH_SIZE, N_FEATURES)

In [ ]:
def inverse_transform(y, n_features):
  # Scaling back the predictions
  # some hacks to make inverse_transform work 
  # create empty table with n_cols fields
  data_like = np.zeros(shape=(len(y), n_features))
  # put the predicted values in the right field
  data_like[:, 0] = y # assuming aqi values are always in first column 
  # inverse transform and then select the right field
  return scaler.inverse_transform(data_like)[:,0]

In [ ]:
y_test_preds = inverse_transform(y_test_preds, 5) # HERE n_features = # of SCALED features 
y_test = inverse_transform(y_test, 5)

In [ ]:
def calculate_metrics(pred, actual, verbose=True):
    result_metrics = {'mae' : mean_absolute_error(pred, actual),
                      'mape' : mean_absolute_percentage_error(pred, actual),
                      'mse' : mean_squared_error(pred, actual), 
                      'rmse' : mean_squared_error(pred, actual) ** 0.5
                      }
    
    if verbose:
      print("Mean Absolute Error:       ", result_metrics["mae"])
      print("Mean Absolute Percentage Error:       ", result_metrics["mape"])
      print("Mean Squared Error:   ", result_metrics["mse"])
      print("Root Mean Squared Error:   ", result_metrics["rmse"])
      
    return result_metrics

In [ ]:
metrics = calculate_metrics(y_test_preds, y_test)

NameError: ignored

In [ ]:
test_times = aqi_daily.iloc[2922+TIMESTEPS:,0]

plt.figure(figsize=(20,5))
plt.plot(test_times, y_test, color = 'red', linewidth=2.0, alpha = 0.6)
plt.plot(test_times, y_test_preds, color = 'blue', linewidth=0.8)
plt.legend(['Actual','Predicted'])
plt.xlabel('Time')
plt.ylabel('AQI values')
plt.title('Test data prediction')
plt.show()

NameError: ignored

<Figure size 1440x360 with 0 Axes>

# Hyperparameter Tuning

In [ ]:
N_FEATURES = train.shape[1]
INPUT_DIM = N_FEATURES
OUTPUT_DIM = 1

In [ ]:
# GRU Hyperparameters
TIMESTEPS = [1, 3, 5, 7, 10]
LEARNING_RATE = [0.001, 0.1]
BATCH_SIZE = [32, 64, 128]
N_EPOCHS = [10, 50, 100]
DROPOUT = [0, 0.1, 0.5, 0.7]
WEIGHT_DECAY = [0] # TODO: research reasonable values 
HIDDEN_DIM = [32, 128] 
LAYER_DIM = [2]

## GRU

In [ ]:
# For loops to implement random search of hyperparameters 
criterion = torch.nn.MSELoss()
errors_df = pd.DataFrame()

for t in TIMESTEPS:
  train_ts = TimeSeries(train, t, N_FEATURES)
  test_ts = TimeSeries(test, t, N_FEATURES) 

  for b in BATCH_SIZE:
    train_loader = DataLoader(train_ts, shuffle=True, batch_size=b)
    test_loader = DataLoader(test_ts, shuffle=True, batch_size=b)

    for l in LEARNING_RATE: 
      for e in N_EPOCHS:
        for d in DROPOUT:
          for w in WEIGHT_DECAY:
            for h in HIDDEN_DIM:
              for lyr in LAYER_DIM: 
                model = GRU(INPUT_DIM, h, lyr, OUTPUT_DIM, d) 
                optimizer = torch.optim.Adam(model.parameters(),lr=l, weight_decay=w)
                train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=e, learning_rate=l, verbose=False)

                y_test_preds, y_test = evaluate(test_loader, b, N_FEATURES)
                y_test_preds = inverse_transform(y_test_preds, 5)
                y_test = inverse_transform(y_test, 5)

                params = { 'timesteps': t, 'batch_size': b, 'learning_rate': l, 'epochs':e, 'dropout': d,
                          'weight_decay' : w, 'hidden_dim': h, 'layer_dim': lyr}
                metrics = calculate_metrics(y_test_preds, y_test, verbose=False)
                out_dict = {**params,  **metrics}

                errors_df = errors_df.append(out_dict, ignore_index=True)


In [ ]:
errors_df = errors_df.sort_values(by="rmse")
errors_df.to_csv('GRU_daily_metrics_ss.csv')
errors_df.head(30)

,timesteps,batch_size,learning_rate,epochs,dropout,weight_decay,hidden_dim,layer_dim,mae,mape,mse,rmse
673,10.0,128.0,0.001,10.0,0.0,0.0,128.0,2.0,9.326026,0.176161,203.803264,14.275968
432,7.0,32.0,0.001,10.0,0.0,0.0,32.0,2.0,9.279652,0.182132,204.028190,14.283844
195,3.0,64.0,0.001,10.0,0.1,0.0,128.0,2.0,9.258016,0.177891,205.408382,14.332075
481,7.0,64.0,0.001,10.0,0.0,0.0,128.0,2.0,9.245228,0.176793,205.572090,14.337785
531,7.0,128.0,0.001,10.0,0.1,0.0,128.0,2.0,9.315992,0.181059,206.358541,14.365185
529,7.0,128.0,0.001,10.0,0.0,0.0,128.0,2.0,9.077035,0.178338,206.532612,14.371243
434,7.0,32.0,0.001,10.0,0.1,0.0,32.0,2.0,9.283469,0.176611,207.016733,14.388076
343,5.0,64.0,0.001,10.0,0.7,0.0,128.0,2.0,9.419908,0.180651,207.228932,14.395448
396,5.0,128.0,0.001,50.0,0.5,0.0,32.0,2.0,9.577021,0.185705,207.230190,14.395492
480,7.0,64.0,0.001,10.0,0.0,0.0,32.0,2.0,9.214243,0.179561,207.634384,14.409524


## CNN-LSTM

In [ ]:
# CNN-LSTM Hyperparameters (same like GRU, except TIMESTEPS)
TIMESTEPS = [7]

In [ ]:
# Additional CNN-LSTM Hyperparameter
N_FILTERS = N_FEATURES 

In [ ]:
!pwd

/content/drive/My Drive/Colab Notebooks/stats207


In [ ]:
import csv

output=open('metrics_output/CNNLSTM_daily_metrics_timestep7.csv', mode='a', newline='')
fieldnames = ['timesteps', 'batch_size', 'learning_rate', 'epochs', 'dropout', 'weight_decay', 'hidden_dim', 'layer_dim', 'window', 'mae', 'mape', 'mse', 'rmse']
output_writer = csv.DictWriter(output, fieldnames=fieldnames, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
output_writer.writeheader()

110

In [ ]:
criterion = torch.nn.MSELoss()
errors_df = pd.DataFrame()

for t in TIMESTEPS:
  train_ts = TimeSeries(train, t, N_FEATURES)
  test_ts = TimeSeries(test, t, N_FEATURES) 
  WINDOW = [ i for i in range(2, t)]
  for b in BATCH_SIZE:
    train_loader = DataLoader(train_ts, shuffle=True, batch_size=b)
    test_loader = DataLoader(test_ts, shuffle=True, batch_size=b)

    for l in LEARNING_RATE: 
      for e in N_EPOCHS:
        for d in DROPOUT:
          for w in WEIGHT_DECAY:
            for h in HIDDEN_DIM:
              for lyr in LAYER_DIM: 
                for win in WINDOW:
                  params = { 'timesteps': t, 'batch_size': b, 'learning_rate': l, 'epochs':e, 'dropout': d,
                            'weight_decay' : w, 'hidden_dim': h, 'layer_dim': lyr, 'window': win}
                  #print(params)

                  model = CNNLSTM(INPUT_DIM, h, lyr, OUTPUT_DIM, d, win, N_FILTERS)
                  optimizer = torch.optim.Adam(model.parameters(),lr=l, weight_decay=w)
                  train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=e, learning_rate=l, verbose=False)

                  y_test_preds, y_test = evaluate(test_loader, b, N_FEATURES)
                  y_test_preds = inverse_transform(y_test_preds, 5) # here N_FEATURES = 5 
                  y_test = inverse_transform(y_test, 5) # here N_FEATURES = 5 

                  
                  metrics = calculate_metrics(y_test_preds, y_test, verbose=False)
                  out_dict = {**params,  **metrics}

                  output_writer.writerow(out_dict)
                  output.flush()
                  errors_df = errors_df.append(out_dict, ignore_index=True)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-53-66e94296c355>", line 25, in <module>
    train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=e, learning_rate=l, verbose=False)
  File "<ipython-input-35-c65644d1a3b3>", line 18, in train_model
    optimizer.step()
  File "/usr/local/lib/python3.8/dist-packages/torch/optim/optimizer.py", line 113, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torch/optim/adam.py", line 157, in step
    adam(params_with_grad,
  File "/usr/local/lib/python3.8/dist-packages/torch/optim/adam.py", line 213, in adam
    func(params,
  File "/usr/local/lib/python3.8/dist-packages/torch/

KeyboardInterrupt: ignored

In [ ]:
output.close()

In [ ]:
errors_df = errors_df.sort_values(by="rmse")
errors_df.to_csv('CNNLSTM_daily_metrics.csv')
errors_df.head(30)